# Opérateurs de requête MongoDB - comparaison: `$eq`, `$ne`, `$gt`, `$gte`, `$lt`, `$lte`, `$in`, `$nin`

Ce notebook présente les opérateurs de comparaison de MongoDB à travers des exemples simples et accessibles.  
L'objectif est de montrer comment filtrer des documents à l'aide de conditions simple de comparaison sur un champ de données dans une base MongoDB.

Nous allons :
- Créer une connexion à MongoDB local (via Docker)
- Insérer des données fictives
- Illustrer les opérateurs de comparaison un par un sur différents types de données

## 1. Connexion à la base de données

Avant toute opération sur une base de données, il est nécessaire de s'y connecter:

In [167]:
from pymongo import MongoClient
import pandas as pd

# Définition du port à utiliser [à changer en fonction la configuration de votre installation locale]
MONGODB_LOCAL_PATH = "mongodb://localhost:27017/"

# Définition de la base de données à utiliser
MONGODB_NAME = "comparison_operator_db"

# Connexion
client = MongoClient(MONGODB_LOCAL_PATH)
db = client[MONGODB_NAME]

## 2. Création d'une collection pour un environnement de test

Les commandes suivantes permettent de créer une collection servant d'environnement de test pour les différentes opérations de comparaison de ce notebook

In [168]:
try:
    # Création de la collection
    db_user = db.create_collection("User")
except:
    # Suppression puis création de la collection
    db.User.drop()
    db_user = db.create_collection("User")

On va remplir cette collection avec un petit nombre de documents

In [169]:
documents = [
    {
        "name": "Paul",
        "age": 32,
        "weight": 79.3,
        "is_employed": True,
        "address": ["France","Nord","Lille"],
    },
    {
        "name": "Maria",
        "age": 52,
        "weight": 65.2,
        "is_employed": True,
        "address": ["France","Gironde","Bordeaux"],
    },
    {
        "name": "James",
        "age": 22,
        "weight": 75.9,
        "is_employed": False,
        "address": ["France","Alpes Maritime","Nice"],
    },
    {
        "name": "Maeva",
        "age": 78,
        "weight": 54.3,
        "is_employed": False,
        "address": ["France","Loiret","Orléans"],
    },
    {
        "name": "Jennifer",
        "age": 16,
        "weight": 59.8,
        "is_employed": False,
        "address": ["Suisse","Neuchâtel","Neuchâtel"],
    },
]
db_user.insert_many(documents)
docs = list(db_user.find())
pd.DataFrame(docs)

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
2,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
3,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"
4,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


## 3. Les opérateurs `$eq` et `$ne`

Les deux premiers opérateurs équivalent à demander l'égalité ou l'inégalité simple dans une requête. Ils s'invoquent via le schéma suivant:

`{ <field>: { $eq: <value> } }`

ou 

`{ <field>: { $ne: <value> } }`

---

### **Exemple sur une chaîne de caractères**

> "Je recherche Paul"

In [170]:
pd.DataFrame(db_user.find({"name": {"$eq": "Paul"}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"


À noter que ceci est équivalent au requêtage simple

In [171]:
pd.DataFrame(db_user.find({"name": "Paul"}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"


Logiquement, l'opérateur `$ne` renvoit les autres documents

In [172]:
pd.DataFrame(db_user.find({"name": {"$ne": "Paul"}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
1,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
2,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"
3,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


### Exemple sur un entier

> "Je recherche la personne qui a 32 ans"

In [173]:
pd.DataFrame(db_user.find({"age": {"$eq": 32}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"


À noter que ceci est équivalent au requêtage simple

In [174]:
pd.DataFrame(db_user.find({"age": 32}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"


Logiquement, l'opérateur `$ne` renvoit les autres documents

In [175]:
pd.DataFrame(db_user.find({"age": {"$ne": 32}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
1,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
2,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"
3,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


### Exemple sur un nombre réel

> "Je recherche la personne qui pèse 79.3 kg"

In [176]:
pd.DataFrame(db_user.find({"weight": {"$eq": 79.3}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"


À noter que ceci est équivalent au requêtage simple

In [177]:
pd.DataFrame(db_user.find({"weight": 79.3}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"


Note: les nombres réelles ont une certaine précision, étant donné la façon dont ils sont encodés en mémoire. Ainsi, la requête suivante renvoit une liste vide...

In [178]:
pd.DataFrame(db_user.find({"weight": 79.3000001}).to_list())

""


... tandis que celle-ci renvoit bien un document

In [179]:
pd.DataFrame(db_user.find({"weight": 79.300000000000001}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"


Logiquement, l'opérateur `$ne` renvoit les autres documents

In [180]:
pd.DataFrame(db_user.find({"weight": {"$ne": 79.3}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
1,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
2,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"
3,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


### Exemple sur un booléen

> "Je recherche les personnes qui sont employés"

In [181]:
pd.DataFrame(db_user.find({"is_employed": {"$eq": True}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"


À noter que ceci est équivalent au requêtage simple

In [182]:
pd.DataFrame(db_user.find({"is_employed": True}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"


Logiquement, l'opérateur `$ne` renvoit les autres documents

In [183]:
pd.DataFrame(db_user.find({"is_employed": {"$ne": True}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
1,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"
2,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


### Exemple sur une liste

In [184]:
pd.DataFrame(db_user.find({"address": {"$eq": ["France", "Nord", "Lille"]}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"


In [185]:
pd.DataFrame(db_user.find({"address": {"$eq": ["France", "Nord"]}}).to_list())

""


À noter que ceci est équivalent au requêtage simple

In [186]:
pd.DataFrame(db_user.find({"address": ["France", "Nord", "Lille"]}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"


Logiquement, l'opérateur `$ne` renvoit les autres documents

In [187]:
pd.DataFrame(db_user.find({"address": {"$ne": ["France", "Nord", "Lille"]}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
1,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
2,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"
3,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


## 4. Les opérateurs `$gt`, `$lt`, `$gte` et `$lte`

Ces opérateurs permettent de faire la comparaison de valeurs en suivant un ordre logique suivant le type de donnée. Ils s'invoquent via le schéma suivant:

`{ <field>: { $gt: <value> } }`

ou 

`{ <field>: { $gte: <value> } }`

ou 

`{ <field>: { $lt: <value> } }`

ou 

`{ <field>: { $lte: <value> } }`

---


## 4.1 Plus grand que `$gt`, plus grand que ou égal à `$gte`

### Exemple sur une chaîne de caractères

> "Je recherche les personnes dont le nom est plus grand que **Maeva** par ordre alphabétique" 

In [188]:
pd.DataFrame(db_user.find({"name": {"$gt": "Maeva"}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"


On peut également inclure **Maeva** au résultat.

In [189]:
pd.DataFrame(db_user.find({"name": {"$gte": "Maeva"}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
2,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"


### Exemple sur un entier

> "Je recherche les personnes qui ont plus que 32 ans"

In [190]:
pd.DataFrame(db_user.find({"age": {"$gt": 32}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
1,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"


On peut également inclure **Paul** au résultat.

In [191]:
pd.DataFrame(db_user.find({"age": {"$gte": 32}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
2,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"


### Exemple sur un nombre réel

> "Je recherche les personnes qui pèsent plus que 65.2 kg"

In [192]:
pd.DataFrame(db_user.find({"weight": {"$gt": 65.2}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"


In [193]:
pd.DataFrame(db_user.find({"weight": {"$gte": 65.2}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
2,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"


### Exemple sur un booléen

In [194]:
pd.DataFrame(db_user.find({"is_employed": {"$gt": False}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"


In [195]:
pd.DataFrame(db_user.find({"is_employed": {"$gte": False}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
2,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
3,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"
4,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


À noter que les booléens étant associé à **False**:**0** et **True**:**1**, l'ordre est logiquement établi ainsi.

### Exemple sur un vecteur

Dans le cas des listes l'opérateur de comparaison est appliqué itérativement sur les éléments de liste.

In [196]:
pd.DataFrame(db_user.find({"address": {"$gt": ["France", "Loiret", "Orléans"]}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


In [197]:
pd.DataFrame(db_user.find({"address": {"$gte": ["France", "Loiret", "Orléans"]}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"
2,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


In [198]:
pd.DataFrame(db_user.find({"address": {"$gt": ["France"]}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
2,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
3,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"
4,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


Dans ce dernier cas, on voit bien que toutes les adresses sont considérées comme plus grande que ```["France"]``` car les second éléments sont comparés à une chaîne de caractères vide.

## 4.2 Plus petit que `$lt`, plus petit que ou égal à `$lte`

On reprend les mêmes exemples que pour le paragraphe précédent mais en utilisant les opérateurs inverses de comparaison.

### Exemple sur une chaîne de caractères

In [199]:
pd.DataFrame(db_user.find({"name": {"$lt": "Maeva"}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
1,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


In [200]:
pd.DataFrame(db_user.find({"name": {"$lte": "Maeva"}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
1,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"
2,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


### Exemple sur un entier

In [201]:
pd.DataFrame(db_user.find({"age": {"$lt": 32}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
1,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


In [202]:
pd.DataFrame(db_user.find({"age": {"$lte": 32}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
2,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


### Exemple sur un nombre réel

In [203]:
pd.DataFrame(db_user.find({"weight": {"$lt": 65.2}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"
1,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


In [204]:
pd.DataFrame(db_user.find({"weight": {"$lte": 65.2}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
1,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"
2,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


### Exemple sur un booléen

In [205]:
pd.DataFrame(db_user.find({"is_employed": {"$lt": False}}).to_list())

""


In [206]:
pd.DataFrame(db_user.find({"is_employed": {"$gte": False}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
2,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
3,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"
4,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


À noter que les booléens étant associé à False:0 et True:1, l'ordre est logiquement établi ainsi.

### Exemple sur un vecteur

In [207]:
pd.DataFrame(db_user.find({"address": {"$lt": ["France", "Loiret", "Orléans"]}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
1,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"


In [208]:
pd.DataFrame(db_user.find({"address": {"$lte": ["France", "Loiret", "Orléans"]}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
1,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
2,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"


In [209]:
pd.DataFrame(db_user.find({"address": {"$lt": ["France"]}}).to_list())

""


## 5. Les opérateurs `$in` et `$nin`

Ces deux opérateurs suivants sont un équivalent à `$eq` et `$ne` mais en permettant de spécifier plusieurs valeurs de référence en même temps. Ils s'invoquent via le schéma suivant:

`{ <field>: { $in: [<value1>, <value2>, ...] } }`

ou 

`{ <field>: { $nin: [<value1>, <value2>, ...] } }`

---


### Exemple sur une chaîne de caractères

> "Je recherche **Paul** et **Maeva**"

In [210]:
pd.DataFrame(db_user.find({"name": {"$in": ["Paul", "Maeva"]}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"


In [211]:
pd.DataFrame(db_user.find({"name": {"$nin": ["Paul", "Maeva"]}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
1,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
2,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


### Exemple sur un entier

> "Je recherche les personnes qui ont 32 ans ou 78 ans."

In [ ]:
pd.DataFrame(db_user.find({"age": {"$in": [32, 78]}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"


In [213]:
pd.DataFrame(db_user.find({"age": {"$nin": [32, 78]}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
1,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
2,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


### Exemple sur un nombre réel

> "Je recherche les personnes qui pèsent 79.3 kg ou 54.3 kg"

In [215]:
pd.DataFrame(db_user.find({"weight": {"$in": [79.3, 54.3]}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"


In [216]:
pd.DataFrame(db_user.find({"weight": {"$nin": [79.3, 54.3]}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
1,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
2,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


### Exemple sur un booléen

> "Je recherche les personnes qui sont employés et qui ne le sont pas"

In [217]:
pd.DataFrame(db_user.find({"is_employed": {"$in": [True, False]}}).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
2,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
3,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"
4,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"


In [218]:
pd.DataFrame(db_user.find({"is_employed": {"$nin": [True, False]}}).to_list())

""


### Exemple sur un vecteur

> "Je recherche les personnes qui vivent à Lille ou à Orléans."

In [ ]:
pd.DataFrame(db_user.find(
    {"address": {
        "$in": [
            ["France", "Nord", "Lille"], 
            ["France", "Loiret", "Orléans"]
        ]
    }}
).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6055,Paul,32,79.3,True,"[France, Nord, Lille]"
1,6887926913b2bc57a40b6058,Maeva,78,54.3,False,"[France, Loiret, Orléans]"


In [223]:
pd.DataFrame(db_user.find(
    {"address": {
        "$nin": [
            ["France", "Nord", "Lille"], 
            ["France", "Loiret", "Orléans"]
        ]
    }}
).to_list())

,_id,name,age,weight,is_employed,address
0,6887926913b2bc57a40b6056,Maria,52,65.2,True,"[France, Gironde, Bordeaux]"
1,6887926913b2bc57a40b6057,James,22,75.9,False,"[France, Alpes Maritime, Nice]"
2,6887926913b2bc57a40b6059,Jennifer,16,59.8,False,"[Suisse, Neuchâtel, Neuchâtel]"
